In [1]:
%pwd

'c:\\Coding\\DataAnalysis'

In [2]:
%ls comento_test_daily_20201127.gz

 C ����̺��� �������� �̸��� �����ϴ�.
 ���� �Ϸ� ��ȣ: BEFE-3882

 c:\Coding\DataAnalysis ���͸�

2021-01-21  ���� 01:55        11,506,437 comento_test_daily_20201127.gz
               1�� ����          11,506,437 ����Ʈ
               0�� ���͸�  31,923,236,864 ����Ʈ ����


In [3]:
%mv

UsageError: Line magic function `%mv` not found.


In [6]:
import pandas as pd
# example_obj=Series?

In [7]:
df=pd.read_csv('comento_test_daily_20201127.gz',compression='gzip',sep='\t',header=None)
df.head()

,0,1,2,3,4,5
0,121.11.xxx.xxx,23d-rmq-holjwfs,1606450990,https://news.sample.comento.com?st=1&media=029...,48,m
1,121.11.xxx.xxx,23o-4oo-uvhg2v4,1606445957,https://news.sample.comento.com?st=1&media=019...,51,m
2,121.11.xxx.xxx,2gc-rkl-mhwiof4,1606461536,https://news.sample.comento.com?st=1&media=009...,35,f
3,121.11.xxx.xxx,2gg-hqp-k3nv7vw,1606461630,https://news.sample.comento.com?st=1&media=029...,19,m
4,121.11.xxx.xxx,2gg-hqp-k3nv7vw,1606462310,https://news.sample.comento.com?st=1&media=079...,19,m


In [110]:
import gzip
import re
import sys
import datetime

cnt=0
m_cnt=0

outfile = open('comento_test_daily_20201127.mapout','w',encoding='UTF-8')
f=gzip.open('comento_test_daily_20201127.gz','r')

for line in f:
    line = (line.decode('utf8').strip())
    fields = line.split('\t')
    
    cnt+=1
    userkey=fields[1]
    url=fields[3]
    timestemp = fields[2]
    # 1.timestemp(epoch time; 1970년 1월 1일 0시 0분 0초부터 밀리세컨드 값으로 나타낸 값)으로 나타나있는 시간을 우리가 볼 수 있는 시간으로 변형 가능. 
    # a=int(timestemp)
    # date=datetime.datetime.fromtimestamp(a).strftime('%Y-%m-%d       %H:%M:%S')
    
    # 2.시간을 밀리세컨트로 나타낸 값.
    # from datetime import datetime
    # import time
    # ts='2021-01-27 13:06:30'
    # timestamp = time.mktime(datetime.strptime(ts, '%Y-%m-%d   %H:%M:%S').timetuple())
    # print(timestamp) 


# from datetime import datetime
# d = datetime.fromtimestamp(time.time())
# print(d)
    url_list=url.split('?')
    if len(url_list)>1:
        params=url_list[1]
        result=re.search('[&]+media=(?P<media>[^&]*)',params)
        if result:
            media=result.group('media')
            m_cnt+=1

            outfile.write(userkey+ '\t' + media + '\t' + timestemp + '\t1\n')
f.close()

In [118]:
#위의 전처리한 데이터를 가지고 새로운 데이터 프레임 만들기
df= pd.read_csv('comento_test_daily_20201127.mapout',sep='\t', names=['userkey','media','timestemp','cnt'])
df_sorted_by_values=df.sort_values(by=['userkey'],axis=0)
df_sorted_by_values.to_csv('comento_test_daily_20201127.reducer_input',index=False, header=None, sep='\t')
df_sorted_by_values

,userkey,media,timestemp,cnt
360000,232-44p-2jqnwfy,1914,1606472053,1
355067,232-44p-2jqnwfy,1914,1606472070,1
288389,232-44p-2jqnwfy,37974,1606472081,1
349933,232-44p-2jqnwfy,909,1606427896,1
9196,232-44p-2jqnwfy,1916,1606456198,1
...,...,...,...,...
259922,zqz-rka-mygh5fy,8981,1606455120,1
290986,zqz-rka-mygh5fy,901,1606437371,1
38869,zqz-ryn-igu2afm,1915,1606474240,1
307456,zqz-ryn-igu2afm,2922,1606488137,1


In [119]:
#reducer 로직 작성
import sys

#30분을 세션타임으로 지정
SESSION_TIME=1800

#init
pre_key=None
pre_ts=0
result_dic={'pv':0,'usercnt':0,'duration':0,'visit':0} #결과 값 컬럼 추가

#reducer 파일 열기
f=open('comento_test_daily_20201127.reducer_input','r')

for line in f:
    fields= line.strip().split('\t')
    userkey, meida, ts, cnt = fields
    key=userkey
    ts=int(ts)
    #만약 pre_key가 key가 아니고, pre_key의 값이 있다면,
    if pre_key != key and pre_key is not None:
        #초기화한 딕셔너리의usercnt라는 키에 +1씩 증가
        result_dic['usercnt']+=1
    pre_key=key
        #init
        #새로운 usercount가 나왔고, 이전 방문시간이 없으니 꼭 pre_ts를 초기화 해줘야 함.
        pre_ts = 0
    #pv는 로그수 임으로 바로 카운트
    result_dic['pv']+=1
    
    
    #duration은 이전 라인과의 시간차를 보고 30분 이내만 더해 줌.
    #첫 라인에서는! pre_ts가 0임으로 무조건 30분 이상 차이가 날 것임으로 더해지지 않음.
    du = ts - pre_ts
    if (du>0 and du<=SESSION_TIME):
        result_dic['duration']+=du
    pre_ts = timestemp
    #visit은 30분 초과의 시간차가 날때 더해준다.
    #첫 라인은 pre_ts가 0이니 무조건 visit + 1이다.
    if (du>SESSION_TIME):
        result_dic['visit']+=1

    
    #media의 갯수를 구하는 코딩
    # if media_dic.get(media) is None:
    #     media_dic[media]={'usercnt':0,'pv':0}
    # media_dic[media]['pv']+=1

    # pre_key=key
    # pre_media = media

f.close()

# 마지막 라인
if pre_key is not None:
    result_dic['usercnt']+=1

print(str(result_dic['usercnt'])+'\t'+str(result_dic['pv'])+'\t'+str(result_dic['visit'])+'\t'+str(result_dic['duration']))
# media에 대한 코딩
# if pre_key is not None:
#     media_dic[pre_media]["usercnt"]+=1
# for _media in media_dic:
#     print(str(_media)+"\t"+str(media_dic[_media]["usercnt"])+'\t'+ str(media_dic[_media]['pv']))

72010	516192	413854	12594540


In [50]:
from datetime import datetime
import time
ts='2021-01-27 13:06:30'
timestamp = time.mktime(datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').timetuple())
print(timestamp) #시간을 밀리세컨트로 나타낸 값.
import datetime 
a=int(timestamp)
date=datetime.datetime.fromtimestamp(a).strftime('%Y-%m-%d %H:%M:%S')
print(date)
from datetime import datetime
d = datetime.fromtimestamp(time.time())
print(d)

2021-01-27 13:06:30


2021-01-28 13:50:13.897310


In [6]:
logic=[
    {
        "IP address":"121.11.xxx.xxx"
        'browser_cookie': '23d-rmq-holjwfs'
        'timestamp': '1606450990'
        'url': 'https://news.sample.comento.com?st=1&media=029'
	    'age': '48'
        'gender': 'm'
        # 'share' :'yes' #공유 링크를 클릭하여 공유를 했는지 여부를         가지고 공유된 ip에 추천 로직도 고려해볼 수 있을까? 
    }
]

SyntaxError: invalid syntax (<ipython-input-6-b76ee4a94bf6>, line 4)

In [7]:
import sys
print(sys.path)

['c:\\Coding\\DataAnalysis', 'c:\\Users\\Administrator\\.vscode\\extensions\\ms-toolsai.jupyter-2020.12.414227025\\pythonFiles', 'c:\\Users\\Administrator\\.vscode\\extensions\\ms-toolsai.jupyter-2020.12.414227025\\pythonFiles\\lib\\python', 'C:\\ProgramData\\Anaconda3\\python37.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', '', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Administrator\\.ipython']


In [3]:
import numpy as np
import pandas as pd


In [5]:
np.random.normal()
pd.Series?

Init signature:
pd.Series(
    data=None,
    index=None,
    dtype=None,
    name=None,
    copy=False,
    fastpath=False,
)
Docstring:     
One-dimensional ndarray with axis labels (including time series).

Labels need not be unique but must be a hashable type. The object
supports both integer- and label-based indexing and provides a host of
methods for performing operations involving the index. Statistical
methods from ndarray have been overridden to automatically exclude
missing data (currently represented as NaN).

Operations between Series (+, -, /, *, **) align values based on their
associated index values-- they need not be the same length. The result
index will be the sorted union of the two indexes.

Parameters
----------
data : array-like, Iterable, dict, or scalar value
    Contains data stored in Series.

    .. versionchanged :: 0.23.0
       If data is a dict, argument order is maintained for Python 3.6
       and later.

index : array-like or Index (1d)
    Values must